In [3]:
import sys, os
sys.path.append(os.path.abspath(".."))
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from src.predictive_models import (
    load_datasets,
    build_frequency_model,
    predict_frequency_model,
    build_sensationalism_model,
    predict_sensationalism_model,
    build_malicious_account_model,
    predict_malicious_account_model,
    build_naive_realism_model,
    predict_naive_realism_model,
)   
from src.articles import (
    fetch_article,
    prepare_article_for_models, 
    train_party_and_job_models
)

# from google import genai
# from google.generativeai import types


# Load Datasets

In [ ]:
sys.path.append(os.path.abspath(".."))
train_path = "../data/train2.tsv"
val_path = "../data/val2.tsv"
test_path = "../data/test2.tsv"
df_train, df_val, df_test = load_datasets(
    train_path, val_path, test_path
) 



# Load Models

### Frequency Heuristic

In [ ]:
train_freq = df_train.copy()
val_freq   = df_val.copy()
test_freq  = df_test.copy()

model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq = build_frequency_model(train_freq)

### Sensationalism

In [ ]:
train_sens = df_train.copy()
val_sens  = df_val.copy()
test_sens = df_test.copy()

sens_pipeline, sens_preproc, sens_meta, sens_num = build_sensationalism_model(train_sens, val_sens, test_sens)


### Malicious Account

In [ ]:
train_mal = df_train.copy()
val_mal   = df_val.copy()
test_mal  = df_test.copy()

model_malicious, tfidf_malicious, le_malicious = build_malicious_account_model(train_mal)


### Naive Realism

In [ ]:
train_nr = df_train.copy()
val_nr   = df_val.copy()
test_nr  = df_test.copy()

naive_pipeline, naive_preproc, naive_meta, naive_num = build_naive_realism_model(train_nr, val_nr, test_nr)


# RAG

In [4]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks, start = [], 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

In [5]:
def setup_rag_for_article(article_df):
    text = article_df["statement"].iloc[0]
    chunks = chunk_text(text)
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(chunks)

    client = chromadb.Client(Settings(anonymized_telemetry=False))
    collection = client.get_or_create_collection(name="articles")
    collection.add(
        ids=[f"chunk_{i}" for i in range(len(chunks))],
        documents=chunks,
        embeddings=embeddings.tolist(),
        metadatas=[{
            "source": article_df["source"].iloc[0],
            "publish_date": article_df["publish_date"].iloc[0]
        } for _ in chunks]
    )
    return collection, embedder

In [6]:
def retrieve_context(query, collection, embedder, n_results=3):
    query_embedding = embedder.encode([query])
    results = collection.query(query_embeddings=query_embedding.tolist(), n_results=n_results)
    retrieved = [doc for doc in results["documents"][0]]
    return "\n\n".join(retrieved)

# Prompting

In [ ]:
load_dotenv() 
GEMINI_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
(party_clf, party_le), (job_clf, job_le) = train_party_and_job_models(df_train)


In [ ]:
cnn_df = prepare_article_for_models(fetch_article("https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis"), job_clf, job_le, party_clf, party_le)
article_text = cnn_df['statement'][0]

In [ ]:
sens_pred = predict_sensationalism_model(cnn_df, sens_pipeline, sens_preproc, sens_meta, sens_num)['predicted_sensationalism'][0]
nr_pred = predict_naive_realism_model(cnn_df, naive_pipeline, naive_preproc, naive_meta, naive_num)['predicted_naive_realism'][0]
ma_pred = predict_malicious_account_model(cnn_df, model_malicious, tfidf_malicious, le_malicious)['predicted_malicious_account'][0]
freq_pred = predict_frequency_model(cnn_df, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq)['predicted_frequency_heuristic'][0]

In [ ]:
prompt = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

You are also given reference model scores from predictive models.
Treat these as informative context, not ground truth, and reason independently.

### Reference Model Scores
- Frequency Heuristic: {freq_pred}
- Malicious Account: {ma_pred}
- Sensationalism: {sens_pred}
- Naive Realism: {nr_pred}

### Article to Evaluate:
{article_text}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

## Different Prompting

In [ ]:
def export_hand_label_template():
    sample = df_val.sample(20, random_state=42)[
        ["id", "statement", "label", "speaker", "party", "context"]
    ]
    sample.to_csv("hand_labels_template.csv", index=False)
    return sample

In [ ]:
export_hand_label_template()

In [ ]:
client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))

cnn_df = prepare_article_for_models(
    fetch_article("https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis")
)
article_text = cnn_df["statement"].iloc[0]

sens_pred = predict_sensationalism_model(
    cnn_df, sens_pipeline, sens_preproc, sens_meta, sens_num
)["predicted_sensationalism"][0]

nr_pred = predict_naive_realism_model(
    cnn_df, naive_pipeline, naive_preproc, naive_meta, naive_num
)["predicted_naive_realism"][0]

ma_pred = predict_malicious_account_model(
    cnn_df, model_malicious, tfidf_malicious, le_malicious
)["predicted_malicious_account"][0]

freq_pred = predict_frequency_model(
    cnn_df, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq
)["predicted_frequency_heuristic"][0]


def run_prompt(prompt_text, json_mode=False):
    contents = [
        types.Content(
            role="user",
            parts=[types.Part(text=prompt_text)],
        ),
    ]

    config_kwargs = {
        "thinking_config": types.ThinkingConfig(
            thinking_budget=2048,
        ),
    }
    if json_mode:
        config_kwargs["response_mime_type"] = "application/json"

    generate_content_config = types.GenerateContentConfig(**config_kwargs)

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents=contents,
        config=generate_content_config,
    )

    print("RAW RESPONSE:\n")
    print(response.text)

    parsed = None
    if json_mode:
        try:
            parsed = json.loads(response.text)
            print("\nParsed JSON keys:", parsed.keys())
        except json.JSONDecodeError as e:
            print("\nJSON parse failed:", e)

    return response.text, parsed

In [ ]:
prompt_baseline = f"""
You are an expert in misinformation and disinformation detection.

Your task is to read the article below and score how strongly it exhibits each of the following factuality factors:

1. Frequency Heuristic (0–2)
   - 0 = little or no repetition of a claim as a shortcut for truth.
   - 1 = some repetition or “people say…” framing.
   - 2 = strong reliance on repetition or popularity to imply truth.

2. Malicious Account (0–2)
   - 0 = credible, professional, or clearly attributed sources.
   - 1 = somewhat suspicious or biased sources, but not clearly malicious.
   - 2 = clearly deceptive, anonymous, or coordinated behavior.

3. Sensationalism (0–2)
   - 0 = neutral, measured tone.
   - 1 = mildly emotional or dramatic, but still grounded.
   - 2 = highly sensationalized, shock-driven language.

4. Naive Realism (0–2)
   - 0 = balanced, acknowledges alternative views.
   - 1 = somewhat one-sided, but not totally closed off.
   - 2 = dogmatic, treats its view as the only rational one.

INSTRUCTIONS:
- Ignore any external models or tools.
- Just use your own reading of the article.
- For each factor:
  - Give a numeric score from 0–2.
  - Provide 2–3 sentences explaining why.

ARTICLE:
{article_text}

OUTPUT FORMAT (plain text is fine for this pass):
- Frequency Heuristic: <score> – <short explanation>
- Malicious Account: <score> – <short explanation>
- Sensationalism: <score> – <short explanation>
- Naive Realism: <score> – <short explanation>
"""

raw_baseline, _ = run_prompt(prompt_baseline, json_mode=False)


In [ ]:
engineering_prompts = []

In [ ]:
# 2.1 – Adds explicit factor headings
engineering_prompts.append(f"""
You are analyzing the same article as before.

For each of the four factuality factors:
1. Frequency Heuristic
2. Malicious Account
3. Sensationalism
4. Naive Realism

Give:
- A short description of how the article behaves along that dimension.
- A score from 0–2.

Be explicit, like:

Factor: Frequency Heuristic
Score: X
Explanation: ...

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng1, parsed_eng1 = run_prompt(engineering_prompts[0], json_mode=False)

In [ ]:
# 2.2 – Require explicit rubric reference
engineering_prompts.append(f"""
Using the definitions and scale below, score the article.

Frequency Heuristic (0–2): repetition and appeal to popularity.
Malicious Account (0–2): malicious or coordinated accounts as source.
Sensationalism (0–2): exaggerated or emotional language.
Naive Realism (0–2): presents its view as the only valid one.

For each factor:
- Mention specific phrases or sections of the article when possible.
- Then give a score.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng2, parsed_eng2 = run_prompt(engineering_prompts[1], json_mode=False)

In [ ]:
# 2.3 – Ask for bullet point evidence
engineering_prompts.append(f"""
For each factor (Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism):

1. List 2–4 bullet points of evidence from the article (paraphrased).
2. Then give a final score 0–2 and a one-sentence reason.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng3, parsed_eng3 = run_prompt(engineering_prompts[2], json_mode=False)

In [ ]:
# 2.4 – Ask for low / medium / high + numeric mapping
engineering_prompts.append(f"""
Evaluate the article along four factors.

For each factor:
- First classify as "low", "medium", or "high".
- Then map that to 0, 1, or 2.
- Explain why.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng4, parsed_eng4 = run_prompt(engineering_prompts[3], json_mode=False)

In [ ]:
# 2.5 – Introduce uncertainty
engineering_prompts.append(f"""
For each factuality factor, provide:
- A 0–2 score.
- A 0–1 confidence value (how certain you are).
- 2–3 sentences explaining the decision.

Factors: Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng5, parsed_eng5 = run_prompt(engineering_prompts[4], json_mode=False)

In [ ]:
# 2.6 – Ask to separate "language" vs "structure"
engineering_prompts.append(f"""
For each factor, structure your answer like this:

- Language cues: ...
- Structural cues: ...
- Score (0–2): ...

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng6, parsed_eng6 = run_prompt(engineering_prompts[5], json_mode=False)

In [ ]:
# 2.7 – Ask to list specific quoted snippets
engineering_prompts.append(f"""
For each factor, do:

1. List up to 3 short snippets (quoted or paraphrased) that support your judgment.
2. Then give a 0–2 score and explain.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng7, parsed_eng7 = run_prompt(engineering_prompts[6], json_mode=False)

In [ ]:
# 2.8 – Ask to explicitly mention ambiguity
engineering_prompts.append(f"""
For each factor:

1. Briefly describe how the article behaves on that factor.
2. Mention any ambiguities or edge cases.
3. Give a final 0–2 score.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng8, parsed_eng8 = run_prompt(engineering_prompts[7], json_mode=False)

In [ ]:
# 2.9 – Ask for compact tabular summary in text
engineering_prompts.append(f"""
Summarize your assessment as a small table in plain text, with columns:

Factor | Score (0–2) | One-sentence justification

Then below the table, optionally add a short paragraph explaining the overall framing.

Factors: Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng9, parsed_eng9 = run_prompt(engineering_prompts[8], json_mode=False)

In [ ]:
# 2.10 – First time requiring JSON
engineering_prompts.append(f"""
You are now required to output a machine-readable JSON object.

Read the article and produce:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "..."
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "..."
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "..."
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "..."
  }}
}}

Only return JSON, no extra text.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng10, parsed_eng10 = run_prompt(engineering_prompts[9], json_mode=True)

In [ ]:
# 2.11 – Same JSON, but with explicit internal thinking
engineering_prompts.append(f"""
Internally, think step-by-step about each factor, but do NOT show your chain-of-thought.
Externally, output ONLY the JSON object in the exact schema below:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "short explanation"
  }}
}}

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng11, parsed_eng11 = run_prompt(engineering_prompts[10], json_mode=True)

In [ ]:
# 2.12 – JSON + hint that model scores exist
engineering_prompts.append(f"""
We will eventually compare your scores to some model outputs,
but for THIS prompt you must ignore any model scores and only use your own judgment.

Internally, think step-by-step. Externally, output ONLY this JSON:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "short explanation"
  }}
}}

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng12, parsed_eng12 = run_prompt(engineering_prompts[11], json_mode=True)

# In Context Learning

In [ ]:
article_link = "https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis"
cnn_df = prepare_article_for_models(
    fetch_article(article_link)
)
article_text = cnn_df["statement"].iloc[0]

In [ ]:
prompt = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Article to Evaluate:
{article_text}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
generate(prompt)

In [ ]:
prompt_icl = f"""
You are an expert in misinformation and disinformation detection, scoring, and ranking.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Examples

**Example 1:**
Article: The Federal Reserve announced a 0.25% interest rate increase following its quarterly meeting. Economists had predicted the move based on recent inflation data.
Frequency Heuristic: 0 — Single statement of fact with proper attribution and no repetitive framing.
Malicious Account: 0 — Credible financial news source citing official Federal Reserve announcement.
Sensationalism: 0 — Neutral, technical language appropriate for economic reporting.
Naive Realism: 0 — Presents information objectively without claiming singular truth.

**Example 2:**
Article: WAKE UP PEOPLE! Everyone knows the government is controlling us through 5G towers! Thousands are saying it online. The mainstream media won't tell you the TRUTH because they're in on it!
Frequency Heuristic: 2 — Heavy appeals to popularity ('everyone knows,' 'thousands are saying') to establish credibility through repetition.
Malicious Account: 2 — Anonymous source spreading debunked conspiracy theories.
Sensationalism: 2 — All-caps words, exclamation marks, fear-mongering language designed to provoke emotional response.
Naive Realism: 2 — Presents conspiracy as absolute truth, dismisses all mainstream sources, and attempts to isolate readers.

**Example 3:**
Article: Is your food secretly KILLING you? Experts reveal the hidden dangers lurking in your kitchen. You won't believe what we found!
Frequency Heuristic: 1 — Uses populist framing ('hidden dangers') which suggests knowledge.
Malicious Account: 1 — Clickbait journalism from established outlet but not malicious.
Sensationalism: 2 — Clickbait with dramatic language and mystery ('you won't believe').
Naive Realism: 1 — Implies hidden truth without presenting the actual complexity of food safety.

**Example 4:**
Article: Tech company announces its AI will revolutionize healthcare and transform diagnosis forever. The CEO called it a 'game-changing breakthrough' during yesterday's product launch.
Frequency Heuristic: 1 — Repeats transformative framing.
Malicious Account: 0 — Corporate PR from legitimate company so promotional but not malicious.
Sensationalism: 2 — Extreme claims ('revolutionize,' 'transform forever,' 'game-changing').
Naive Realism: 1 — Presents corporate claims as reality without acknowledging uncertainty or limitations.

**Example 5:**
Article: According to multiple sources familiar with the matter, the investigation is ongoing. Officials declined to comment on specifics, citing the active nature of the case.
Frequency Heuristic: 0 — Reports lack of information honestly without speculation.
Malicious Account: 0 — Standard practice of protecting sources while noting limitations.
Sensationalism: 0 — Neutral tone and restrained language.
Naive Realism: 0 — Explicitly communicates uncertainty and ongoing nature of situation.

**Example 6:**
Article: 'Many people are saying the new policy is unfair. It's becoming clear that something needs to change. More and more citizens are questioning the decision every day.'
Frequency Heuristic: 2 — Repeating claims of rising support that lack clear attribution.
Malicious Account: 1 — Lacks verification.
Sensationalism: 1 — Emotionally charged framing ('unfair') and builds urgency but not extreme language.
Naive Realism: 1 — Implies growing consensus makes position correct without presenting other arguments.

**Example 7:**
Article: The new immigration bill sparked heated debate. Republican lawmakers argue it strengthens border security, while Democratic representatives contend it lacks humanitarian protections. Legal experts are divided on constitutional questions.
Frequency Heuristic: 0 — Multiple perspective stated once with attribution to distinct groups.
Malicious Account: 0 — Mainstream news outlet citing political figures and experts.
Sensationalism: 1 — Word 'heated' adds mild emotional tone but otherwise is balanced.
Naive Realism: 0 — Acknowledges multiple perspectives across parties and expert opinion.

**Example 8:**
Article: Shocking allegations have emerged against the senator. Anonymous sources describe a pattern of misconduct, though the senator's team calls the claims 'politically motivated lies' and notes no formal charges have been filed.
Frequency Heuristic: 1 — 'Pattern of misconduct' suggests multiple mentions.
Malicious Account: 1 — Anonymous sourcing.
Sensationalism: 2 — 'Shocking allegations' is designed to attract attention.
Naive Realism: 0 — Presents both accusation and denial. Also acknowledges lack of formal charges.

**Example 9:**
Article: As commonly understood by most experts in the field, renewable energy will replace fossil fuels within two decades. Some outlier researchers dispute this timeline, but the consensus is clear.
Frequency Heuristic: 1 — Frames majority view as established fact.
Malicious Account: 0 — Appears to be legitimate climate journalism citing expert community.
Sensationalism: 0 — Factual tone without emotional manipulation or exaggeration.
Naive Realism: 2 — Dismisses dissenting views and presents prediction as inevitable.

**Example 10:**
Article: Influencer @TrendyLifestyle123 claims this detox tea cured her chronic illness. The product has 50K followers and hundreds of testimonials, though medical professionals warn these supplements are unregulated.
Frequency Heuristic: 1 — Implies truth through follower count and testimonials, but includes medical warning.
Malicious Account: 2 — Suspicious account name and health claims.
Sensationalism: 1 — 'Cured chronic illness' is dramatic claim but is presented as a quote from influencer.
Naive Realism: 1 — Amplifies an unverified claim but includes expert skepticism.

### Article to Evaluate:
{article_text}


### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
generate(prompt_icl)

# Predictive Model Function Calling

In [ ]:
article_link = "https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis"
cnn_df = prepare_article_for_models(
    fetch_article(article_link)
)
article_text = cnn_df["statement"].iloc[0]

In [ ]:
prompt_tool = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Examples
**Example 1:**
Article: The Federal Reserve announced a 0.25% interest rate increase following its quarterly meeting. Economists had predicted the move based on recent inflation data.
Frequency Heuristic: 0 — Single statement of fact with proper attribution and no repetitive framing.
Malicious Account: 0 — Credible financial news source citing official Federal Reserve announcement.
Sensationalism: 0 — Neutral, technical language appropriate for economic reporting.
Naive Realism: 0 — Presents information objectively without claiming singular truth.

**Example 2:**
Article: WAKE UP PEOPLE! Everyone knows the government is controlling us through 5G towers! Thousands are saying it online. The mainstream media won't tell you the TRUTH because they're in on it!
Frequency Heuristic: 2 — Heavy appeals to popularity ('everyone knows,' 'thousands are saying') to establish credibility through repetition.
Malicious Account: 2 — Anonymous source spreading debunked conspiracy theories.
Sensationalism: 2 — All-caps words, exclamation marks, fear-mongering language designed to provoke emotional response.
Naive Realism: 2 — Presents conspiracy as absolute truth, dismisses all mainstream sources, and attempts to isolate readers.

**Example 3:**
Article: Is your food secretly KILLING you? Experts reveal the hidden dangers lurking in your kitchen. You won't believe what we found!
Frequency Heuristic: 1 — Uses populist framing ('hidden dangers') which suggests knowledge.
Malicious Account: 1 — Clickbait journalism from established outlet but not malicious.
Sensationalism: 2 — Clickbait with dramatic language and mystery ('you won't believe').
Naive Realism: 1 — Implies hidden truth without presenting the actual complexity of food safety.

**Example 4:**
Article: Tech company announces its AI will revolutionize healthcare and transform diagnosis forever. The CEO called it a 'game-changing breakthrough' during yesterday's product launch.
Frequency Heuristic: 1 — Repeats transformative framing.
Malicious Account: 0 — Corporate PR from legitimate company so promotional but not malicious.
Sensationalism: 2 — Extreme claims ('revolutionize,' 'transform forever,' 'game-changing').
Naive Realism: 1 — Presents corporate claims as reality without acknowledging uncertainty or limitations.

**Example 5:**
Article: According to multiple sources familiar with the matter, the investigation is ongoing. Officials declined to comment on specifics, citing the active nature of the case.
Frequency Heuristic: 0 — Reports lack of information honestly without speculation.
Malicious Account: 0 — Standard practice of protecting sources while noting limitations.
Sensationalism: 0 — Neutral tone and restrained language.
Naive Realism: 0 — Explicitly communicates uncertainty and ongoing nature of situation.

**Example 6:**
Article: 'Many people are saying the new policy is unfair. It's becoming clear that something needs to change. More and more citizens are questioning the decision every day.'
Frequency Heuristic: 2 — Repeating claims of rising support that lack clear attribution.
Malicious Account: 1 — Lacks verification.
Sensationalism: 1 — Emotionally charged framing ('unfair') and builds urgency but not extreme language.
Naive Realism: 1 — Implies growing consensus makes position correct without presenting other arguments.

**Example 7:**
Article: The new immigration bill sparked heated debate. Republican lawmakers argue it strengthens border security, while Democratic representatives contend it lacks humanitarian protections. Legal experts are divided on constitutional questions.
Frequency Heuristic: 0 — Multiple perspective stated once with attribution to distinct groups.
Malicious Account: 0 — Mainstream news outlet citing political figures and experts.
Sensationalism: 1 — Word 'heated' adds mild emotional tone but otherwise is balanced.
Naive Realism: 0 — Acknowledges multiple perspectives across parties and expert opinion.

**Example 8:**
Article: Shocking allegations have emerged against the senator. Anonymous sources describe a pattern of misconduct, though the senator's team calls the claims 'politically motivated lies' and notes no formal charges have been filed.
Frequency Heuristic: 1 — 'Pattern of misconduct' suggests multiple mentions.
Malicious Account: 1 — Anonymous sourcing.
Sensationalism: 2 — 'Shocking allegations' is designed to attract attention.
Naive Realism: 0 — Presents both accusation and denial. Also acknowledges lack of formal charges.


**Example 9:**
Article: As commonly understood by most experts in the field, renewable energy will replace fossil fuels within two decades. Some outlier researchers dispute this timeline, but the consensus is clear.
Frequency Heuristic: 1 — Frames majority view as established fact.
Malicious Account: 0 — Appears to be legitimate climate journalism citing expert community.
Sensationalism: 0 — Factual tone without emotional manipulation or exaggeration.
Naive Realism: 2 — Dismisses dissenting views and presents prediction as inevitable.

**Example 10:**
Article: Influencer @TrendyLifestyle123 claims this detox tea cured her chronic illness. The product has 50K followers and hundreds of testimonials, though medical professionals warn these supplements are unregulated.
Frequency Heuristic: 1 — Implies truth through follower count and testimonials, but includes medical warning.
Malicious Account: 2 — Suspicious account name and health claims.
Sensationalism: 1 — 'Cured chronic illness' is dramatic claim but is presented as a quote from influencer.
Naive Realism: 1 — Amplifies an unverified claim but includes expert skepticism.

### Article to Evaluate:
{article_text}

### Tool Usage Requirement:
You also have access to tools that return a predictive model score for each factuality factor.
Treat these model scores as informative context, NOT ground truth. You must reason independently.
Before generating your final JSON answer, you must call all four tools using the article_url provided.
Use this URL when calling the tools: {article_link}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Use your independent analysis of the article and the tool outputs to provide a numeric score and a justification for why that score was chosen. If your score is different than the predictive model score, you must explain why you disagree.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
def run_frequency_model(article_url: str):
    """
    Given an article URL, this tool computes a score for the level of
    frequency heuristic exhibited in the article (0-2)

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2

    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_frequency_model(
        df, model_freq, tfidf_freq, count_vec_freq,
        token_dict_freq, buzzwords_freq, le_freq
    )

    score = int(out["predicted_frequency_heuristic"].iloc[0])
    return {"frequency_heuristic": score}


In [ ]:
def run_sensationalism_model(article_url: str):
    """
    Given an article URL, this tool computes sensationalism level (0–2).

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2
    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_sensationalism_model(
        df, sens_pipeline, sens_preproc, sens_meta, sens_num
    )

    score = int(out["predicted_sensationalism"].iloc[0])
    return {"sensationalism": score}


In [ ]:
def run_malicious_account_model(article_url: str):
    """
    Given an article url, this tool computes a score for the level of
    malicious account this article exhibits (0-2).

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2
    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_malicious_account_model(
        df, model_malicious, tfidf_malicious, le_malicious
    )

    score = int(out["predicted_malicious_account"].iloc[0])
    return {"malicious_account": score}


In [ ]:
def run_naive_realism_model(article_url: str):
    """
    Given an article, this tool scores the degree to which the article presents
    its viewpoint as the only rational truth (0–2).

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2
    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_naive_realism_model(
        df, naive_pipeline, naive_preproc, naive_meta, naive_num
    )

    score = int(out["predicted_naive_realism"].iloc[0])
    return {"naive_realism": score}


In [ ]:
function_map = {
    "run_frequency_model": run_frequency_model,
    "run_sensationalism_model": run_sensationalism_model,
    "run_malicious_account_model": run_malicious_account_model,
    "run_naive_realism_model": run_naive_realism_model,
}


In [ ]:
def generate_with_tools(prompt, article_url, max_turns=5):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )
    tools = [
        types.Tool(
            function_declarations=[
                types.FunctionDeclaration(
                    name="run_frequency_model",
                    description="Returns frequency heuristic score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                ),
                types.FunctionDeclaration(
                    name="run_sensationalism_model",
                    description="Returns sensationalism score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                ),
                types.FunctionDeclaration(
                    name="run_malicious_account_model",
                    description="Returns malicious account score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                ),
                types.FunctionDeclaration(
                    name="run_naive_realism_model",
                    description="Returns naive realism score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                )
            ]
        )
    ]

    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=prompt)]
        )
    ]

    for turn in range(max_turns):
        response = client.models.generate_content(
            model="gemini-2.5-pro",
            contents=contents,
            config=types.GenerateContentConfig(
                tools=tools,
                tool_config=types.ToolConfig(
                    function_calling_config=types.FunctionCallingConfig(
                        mode="AUTO"
                    )
                )
            )
        )
        if not response.candidates:
            break

        candidate = response.candidates[0]
        if not candidate.content or not candidate.content.parts:
            break

        function_calls = []
        text_parts = []

        for part in candidate.content.parts:
            if part.function_call is not None:
                function_calls.append(part.function_call)
            if part.text is not None and part.text.strip():
                text_parts.append(part.text)

        if function_calls:
            contents.append(candidate.content)
            tool_response_parts = []
            for fc in function_calls:
                args = dict(fc.args)
                try:
                    result = function_map[fc.name](**args)
                    tool_response_parts.append(
                        types.Part.from_function_response(
                            name=fc.name,
                            response=result
                        )
                    )
                except Exception as e:
                    import traceback
                    traceback.print_exc()
                    tool_response_parts.append(
                        types.Part.from_function_response(
                            name=fc.name,
                            response={"error": str(e)}
                        )
                    )

            contents.append(
                types.Content(
                    role="tool",
                    parts=tool_response_parts
                )
            )
            continue

        if text_parts:
            final_text = "".join(text_parts)
            print(final_text)
            return final_text
        break

result = generate_with_tools(prompt_tool, article_link)

# Collected Ground Truth

In [ ]:
ground_truth = pd.DataFrame(columns = ['statement', 'frequency_heuristic', 'malicious_account', 'sensationalism', 'naive_realism'])

In [ ]:
def article_ground_truth(article_link, freq_score, ma_score, sens_score, nr_score):
  article_df = prepare_article_for_models(fetch_article(article_link))
  article_text = article_df['statement'][0]

  ground_truth_df = pd.DataFrame([{
      'statement': article_text,
      'frequency_heuristic': freq_score,
      'malicious_account': ma_score,
      'sensationalism': sens_score,
      'naive_realism': nr_score,
      'link': article_link
  }])

  return ground_truth_df



In [ ]:
cnn_link = 'https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis'
cnn_scores = article_ground_truth(cnn_link, 2,0,2,2)
cnn_text = cnn_scores['statement'][0]

In [ ]:
abc7_link = 'https://abc7.com/post/trump-is-committed-giving-americans-2000-tariff-checks-white-house/18149579/'
abc7_scores = article_ground_truth(abc7_link,0,0,0,1)
abc7_text = abc7_scores['statement'][0]

In [ ]:
guardian_link = 'https://www.theguardian.com/us-news/2025/nov/12/house-bill-government-shutdown-vote'
guardian_scores = article_ground_truth(guardian_link, 0,0,0,1)
guardian_text = guardian_scores['statement'][0]

In [ ]:
apn_link = 'https://apnews.com/article/redistricting-gerrymandering-congress-trump-0af8561b1670032fae3e1d2aec7905f0'
apn_scores = article_ground_truth(apn_link, 0,0,0,0)
apn_text = apn_scores['statement'][0]

In [ ]:
act_link = 'https://azcapitoltimes.com/news/2025/11/12/prop-50-exposes-deepening-political-and-geographical-shifts-in-california/'
act_scores = article_ground_truth(act_link,1,0,1,0)
act_text = act_scores['statement'][0]

In [ ]:
ground_truth = pd.concat([ground_truth, cnn_scores, abc7_scores, guardian_scores, apn_scores, act_scores])

In [ ]:
# next: zero shot learning, many shot learning
# in context learning - tuning your models to the functions we're using
# operational definitions for how to measure the different factuality factors
# create the 20 hand done labels
# human readibility - human in the loop feature; no json output
# "im not college educated but im trusting that these students are well versed in these areas as students at UCSD" - thinking in terms of the user
# give percentage of confidence - different metric that we'll be studying as well
#